In [ ]:
# Setup: Import libraries and define simulation framework
import sys
from pathlib import Path

# Add src to path
project_root = Path.cwd()
if project_root.name == "notebooks":
    project_root = project_root.parent
sys.path.insert(0, str(project_root / "src"))

# Force reload of modules to pick up latest changes
import importlib

if "options_hedge" in sys.modules:
    import options_hedge.analyzer
    import options_hedge.strategies
    import options_hedge.vix_floor_lp

    importlib.reload(options_hedge.vix_floor_lp)
    importlib.reload(options_hedge.strategies)
    importlib.reload(options_hedge.analyzer)


import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from options_hedge.analyzer import PortfolioAnalyzer
from options_hedge.market import Market
from options_hedge.portfolio import Portfolio
from options_hedge.simulation import run_simulation
from options_hedge.strategies import (
    conditional_hedging_strategy,
    quarterly_protective_put_strategy,
    vix_ladder_strategy,
)

# Simulation parameters
initial_value = 1_000_000  # $1M starting portfolio
print("✓ Libraries loaded (modules reloaded)")
print(f"✓ Initial portfolio value: ${initial_value:,}")
print("✓ Strategies: Quarterly, Conditional, VIX-Ladder LP")
print("✓ PortfolioAnalyzer ready")

## 📉 Market Crash Simulations

Testing VIX-Ladder LP strategy across three major crashes with different characteristics:
1. **Dot-Com Bubble (1999-2003)**: Prolonged 3-year decline, -49% drawdown
2. **Global Financial Crisis (2007-2009)**: Deep financial crisis, -57% drawdown  
3. **COVID-19 (2020)**: Fastest crash ever (33 days), -34% drawdown


## 📊 Portfolio Evaluation Metrics

We'll use these metrics to compare strategies:
1. **Beta (β)**: Systematic risk - $\beta = \frac{\text{Cov}(R_p, R_m)}{\text{Var}(R_m)}$
2. **Upside/Downside Capture**: Performance in up/down markets
3. **Sortino Ratio**: Risk-adjusted return using downside deviation - $\text{Sortino} = \frac{R_p - R_f}{\sigma_{\text{downside}}}$
4. **Calmar Ratio**: Return vs maximum drawdown - $\text{Calmar} = \frac{\text{Annualized Return}}{\text{Max Drawdown}}$

In [ ]:
# Dot-Com Bubble (1999-2003): Slow crash over 3+ years
market_dotcom = Market(
    ticker="^GSPC", start="1999-01-01", end="2003-12-31", fetch_vix=True
)

portfolio_unhedged_dc = Portfolio(initial_value=initial_value, beta=1.0)
portfolio_quarterly_dc = Portfolio(initial_value=initial_value, beta=1.0)
portfolio_conditional_dc = Portfolio(initial_value=initial_value, beta=1.0)
portfolio_ladder_dc = Portfolio(initial_value=initial_value, beta=1.0)

results_unhedged_dc = run_simulation(
    market_dotcom, portfolio_unhedged_dc, lambda p, price, date, params, m: None, {}
)
results_quarterly_dc = run_simulation(
    market_dotcom, portfolio_quarterly_dc, quarterly_protective_put_strategy, {}
)
results_conditional_dc = run_simulation(
    market_dotcom, portfolio_conditional_dc, conditional_hedging_strategy, {}
)

ladder_params_dc = {
    "hedge_interval": 7,
    "expiry_days": 90,
    "alpha": 0.05,
    "ladder_budget_allocations": [
        (0.05, 0.15, 0.05),
        (0.15, 0.25, 0.15),
        (0.25, 0.40, 0.30),
        (0.40, 1.00, 0.50),
    ],
    "strike_density": 0.05,
    "transaction_cost_rate": 0.05,
}
results_ladder_dc = run_simulation(
    market_dotcom, portfolio_ladder_dc, vix_ladder_strategy, ladder_params_dc
)

print(
    f"Dot-Com: Unhedged {results_unhedged_dc['Value'].iloc[-1]:,.0f} | Quarterly {results_quarterly_dc['Value'].iloc[-1]:,.0f} | Conditional {results_conditional_dc['Value'].iloc[-1]:,.0f} | VIX-Ladder {results_ladder_dc['Value'].iloc[-1]:,.0f}"
)

In [ ]:
# Global Financial Crisis (2007-2009): Deep financial crisis
market_gfc = Market(
    ticker="^GSPC", start="2007-01-01", end="2009-12-31", fetch_vix=True
)

portfolio_unhedged_gfc = Portfolio(initial_value=initial_value, beta=1.0)
portfolio_quarterly_gfc = Portfolio(initial_value=initial_value, beta=1.0)
portfolio_conditional_gfc = Portfolio(initial_value=initial_value, beta=1.0)
portfolio_ladder_gfc = Portfolio(initial_value=initial_value, beta=1.0)

results_unhedged_gfc = run_simulation(
    market_gfc, portfolio_unhedged_gfc, lambda p, price, date, params, m: None, {}
)
results_quarterly_gfc = run_simulation(
    market_gfc, portfolio_quarterly_gfc, quarterly_protective_put_strategy, {}
)
results_conditional_gfc = run_simulation(
    market_gfc, portfolio_conditional_gfc, conditional_hedging_strategy, {}
)

ladder_params_gfc = {
    "hedge_interval": 7,
    "expiry_days": 90,
    "alpha": 0.05,
    "ladder_budget_allocations": [
        (0.05, 0.15, 0.05),
        (0.15, 0.25, 0.15),
        (0.25, 0.40, 0.30),
        (0.40, 1.00, 0.50),
    ],
    "strike_density": 0.05,
    "transaction_cost_rate": 0.05,
}
results_ladder_gfc = run_simulation(
    market_gfc, portfolio_ladder_gfc, vix_ladder_strategy, ladder_params_gfc
)

print(
    f"GFC: Unhedged {results_unhedged_gfc['Value'].iloc[-1]:,.0f} | Quarterly {results_quarterly_gfc['Value'].iloc[-1]:,.0f} | Conditional {results_conditional_gfc['Value'].iloc[-1]:,.0f} | VIX-Ladder {results_ladder_gfc['Value'].iloc[-1]:,.0f}"
)

In [ ]:
# COVID-19 Crash (2019-2021): Fastest crash in history
market_covid = Market(
    ticker="^GSPC", start="2019-06-01", end="2021-06-30", fetch_vix=True
)

portfolio_unhedged_covid = Portfolio(initial_value=initial_value, beta=1.0)
portfolio_quarterly_covid = Portfolio(initial_value=initial_value, beta=1.0)
portfolio_conditional_covid = Portfolio(initial_value=initial_value, beta=1.0)
portfolio_ladder_covid = Portfolio(initial_value=initial_value, beta=1.0)

results_unhedged_covid = run_simulation(
    market_covid, portfolio_unhedged_covid, lambda p, price, date, params, m: None, {}
)
results_quarterly_covid = run_simulation(
    market_covid, portfolio_quarterly_covid, quarterly_protective_put_strategy, {}
)
results_conditional_covid = run_simulation(
    market_covid, portfolio_conditional_covid, conditional_hedging_strategy, {}
)

ladder_params_covid = {
    "hedge_interval": 7,
    "expiry_days": 90,
    "alpha": 0.05,
    "ladder_budget_allocations": [
        (0.05, 0.15, 0.05),
        (0.15, 0.25, 0.15),
        (0.25, 0.40, 0.30),
        (0.40, 1.00, 0.50),
    ],
    "strike_density": 0.05,
    "transaction_cost_rate": 0.05,
}
results_ladder_covid = run_simulation(
    market_covid, portfolio_ladder_covid, vix_ladder_strategy, ladder_params_covid
)

print(
    f"COVID: Unhedged {results_unhedged_covid['Value'].iloc[-1]:,.0f} | Quarterly {results_quarterly_covid['Value'].iloc[-1]:,.0f} | Conditional {results_conditional_covid['Value'].iloc[-1]:,.0f} | VIX-Ladder {results_ladder_covid['Value'].iloc[-1]:,.0f}"
)

## 🎯 Final Analysis: VIX-Ladder LP Strategy

### Simplified Project Scope
We're focusing on **VIX-Ladder LP** as our core contribution, with honest assessment of strengths and limitations.


In [ ]:
def comprehensive_analysis(results_dict, crash_name=""):
    """Analyze strategy performance with focus on volatility and drawdown metrics."""
    print(f"\n{'=' * 80}")
    print(f"STRATEGY PERFORMANCE ANALYSIS: {crash_name}")
    print(f"{'=' * 80}\n")

    metrics = {}
    for name, results in results_dict.items():
        values = results["Value"]
        returns = values.pct_change().dropna()

        # Core metrics
        total_return = (values.iloc[-1] / values.iloc[0] - 1) * 100
        ann_return = (
            (values.iloc[-1] / values.iloc[0]) ** (252 / len(values)) - 1
        ) * 100

        # Risk metrics
        daily_vol = returns.std() * np.sqrt(252) * 100
        max_dd = (
            (values - values.expanding(min_periods=1).max())
            / values.expanding(min_periods=1).max()
        ).min() * 100

        # Downside metrics
        downside_returns = returns[returns < 0]
        downside_vol = (
            downside_returns.std() * np.sqrt(252) * 100
            if len(downside_returns) > 0
            else 0
        )

        # Sharpe (0% risk-free rate)
        sharpe = ann_return / daily_vol if daily_vol != 0 else 0

        metrics[name] = {
            "Total Return (%)": total_return,
            "Annual Return (%)": ann_return,
            "Max Drawdown (%)": max_dd,
            "Daily Vol (%)": daily_vol,
            "Downside Vol (%)": downside_vol,
            "Sharpe Ratio": sharpe,
            "Final Value ($)": values.iloc[-1],
        }

    # Display results
    df = pd.DataFrame(metrics).T
    print(df.to_string())

    # Ladder vs baselines comparison
    if "Unhedged" in metrics and "VIX-Ladder" in metrics:
        print(f"\n{'=' * 80}")
        print("VIX-LADDER LP vs BASELINES")
        print(f"{'=' * 80}\n")

        ladder = metrics["VIX-Ladder"]
        uh = metrics["Unhedged"]

        vol_reduction = (
            (ladder["Daily Vol (%)"] - uh["Daily Vol (%)"]) / uh["Daily Vol (%)"]
        ) * 100
        dd_improvement = ladder["Max Drawdown (%)"] - uh["Max Drawdown (%)"]
        return_cost = ladder["Total Return (%)"] - uh["Total Return (%)"]

        print("vs Unhedged:")
        print(
            f"  • Volatility: {vol_reduction:+.1f}% ({ladder['Daily Vol (%)']:.1f}% vs {uh['Daily Vol (%)']:.1f}%)"
        )
        print(
            f"  • Max Drawdown: {dd_improvement:+.1f}pp ({ladder['Max Drawdown (%)']:.1f}% vs {uh['Max Drawdown (%)']:.1f}%)"
        )
        print(
            f"  • Return Drag: {return_cost:+.1f}pp ({ladder['Total Return (%)']:.1f}% vs {uh['Total Return (%)']:.1f}%)"
        )

        if "Quarterly" in metrics:
            qtr = metrics["Quarterly"]
            print("\nvs Quarterly:")
            print(
                f"  • Max Drawdown: {(ladder['Max Drawdown (%)'] - qtr['Max Drawdown (%)']):+.1f}pp ({ladder['Max Drawdown (%)']:.1f}% vs {qtr['Max Drawdown (%)']:.1f}%)"
            )
            print(
                f"  • Return: {(ladder['Total Return (%)'] - qtr['Total Return (%)']):+.1f}pp ({ladder['Total Return (%)']:.1f}% vs {qtr['Total Return (%)']:.1f}%)"
            )

        if "Conditional" in metrics:
            cond = metrics["Conditional"]
            print("\nvs Conditional:")
            print(
                f"  • Max Drawdown: {(ladder['Max Drawdown (%)'] - cond['Max Drawdown (%)']):+.1f}pp ({ladder['Max Drawdown (%)']:.1f}% vs {cond['Max Drawdown (%)']:.1f}%)"
            )
            print(
                f"  • Return: {(ladder['Total Return (%)'] - cond['Total Return (%)']):+.1f}pp ({ladder['Total Return (%)']:.1f}% vs {cond['Total Return (%)']:.1f}%)"
            )

    return df


# Analyze each crash period
print("\n" + "=" * 80)
print("COMPREHENSIVE ANALYSIS ACROSS CRASH PERIODS")
print("=" * 80)

print("\n\n### DOT-COM BUBBLE (1999-2003)")
dc_analysis = comprehensive_analysis(
    {
        "Unhedged": results_unhedged_dc,
        "Quarterly": results_quarterly_dc,
        "Conditional": results_conditional_dc,
        "VIX-Ladder": results_ladder_dc,
    },
    "Dot-Com Crash",
)

print("\n\n### GLOBAL FINANCIAL CRISIS (2007-2009)")
gfc_analysis = comprehensive_analysis(
    {
        "Unhedged": results_unhedged_gfc,
        "Quarterly": results_quarterly_gfc,
        "Conditional": results_conditional_gfc,
        "VIX-Ladder": results_ladder_gfc,
    },
    "Global Financial Crisis",
)

print("\n\n### COVID-19 CRASH (2019-2021)")
covid_analysis = comprehensive_analysis(
    {
        "Unhedged": results_unhedged_covid,
        "Quarterly": results_quarterly_covid,
        "Conditional": results_conditional_covid,
        "VIX-Ladder": results_ladder_covid,
    },
    "COVID-19",
)

In [ ]:
# Visual Summary: Strategy Performance Comparison
fig, axes = plt.subplots(2, 3, figsize=(18, 10))

crashes = [
    (
        "Dot-Com (1999-2003)",
        results_unhedged_dc,
        results_quarterly_dc,
        results_conditional_dc,
        results_ladder_dc,
    ),
    (
        "GFC (2007-2009)",
        results_unhedged_gfc,
        results_quarterly_gfc,
        results_conditional_gfc,
        results_ladder_gfc,
    ),
    (
        "COVID-19 (2019-2021)",
        results_unhedged_covid,
        results_quarterly_covid,
        results_conditional_covid,
        results_ladder_covid,
    ),
]

for idx, (crash_name, unhedged, quarterly, conditional, ladder) in enumerate(crashes):
    # Top row: Portfolio value over time
    ax = axes[0, idx]
    ax.plot(unhedged.index, unhedged["Value"], label="Unhedged", linewidth=2, alpha=0.7)
    ax.plot(
        quarterly.index,
        quarterly["Value"],
        label="Quarterly",
        linewidth=2,
        alpha=0.7,
        linestyle="--",
    )
    ax.plot(
        conditional.index,
        conditional["Value"],
        label="Conditional",
        linewidth=2,
        alpha=0.7,
        linestyle=":",
    )
    ax.plot(
        ladder.index, ladder["Value"], label="VIX-Ladder LP", linewidth=2.5, alpha=0.9
    )
    ax.set_title(f"{crash_name}\nPortfolio Value", fontsize=12, fontweight="bold")
    ax.set_ylabel("Value ($)")
    ax.legend(loc="best", fontsize=9)
    ax.grid(True, alpha=0.3)
    ax.ticklabel_format(style="plain", axis="y")

    # Bottom row: Drawdown comparison
    ax = axes[1, idx]
    unhedged_dd = (
        (unhedged["Value"] - unhedged["Value"].expanding(min_periods=1).max())
        / unhedged["Value"].expanding(min_periods=1).max()
        * 100
    )
    quarterly_dd = (
        (quarterly["Value"] - quarterly["Value"].expanding(min_periods=1).max())
        / quarterly["Value"].expanding(min_periods=1).max()
        * 100
    )
    conditional_dd = (
        (conditional["Value"] - conditional["Value"].expanding(min_periods=1).max())
        / conditional["Value"].expanding(min_periods=1).max()
        * 100
    )
    ladder_dd = (
        (ladder["Value"] - ladder["Value"].expanding(min_periods=1).max())
        / ladder["Value"].expanding(min_periods=1).max()
        * 100
    )

    ax.plot(unhedged.index, unhedged_dd, linewidth=1.5, label="Unhedged", alpha=0.7)
    ax.plot(
        quarterly.index,
        quarterly_dd,
        linewidth=1.5,
        label="Quarterly",
        alpha=0.7,
        linestyle="--",
    )
    ax.plot(
        conditional.index,
        conditional_dd,
        linewidth=1.5,
        label="Conditional",
        alpha=0.7,
        linestyle=":",
    )
    ax.plot(ladder.index, ladder_dd, linewidth=2, label="VIX-Ladder LP", alpha=0.9)
    ax.fill_between(ladder.index, ladder_dd, 0, alpha=0.2)
    ax.set_title("Drawdown (%)", fontsize=11)
    ax.set_ylabel("Drawdown (%)")
    ax.set_xlabel("Date")
    ax.legend(loc="lower left", fontsize=9)
    ax.grid(True, alpha=0.3)

plt.suptitle(
    "Hedging Strategy Comparison: VIX-Ladder LP vs Baselines",
    fontsize=16,
    fontweight="bold",
    y=1.00,
)
plt.tight_layout()
plt.show()

## 📈 Quantitative Performance Metrics

Now let's analyze each crash using industry-standard metrics:
- **Beta**: Systematic risk vs market
- **Upside/Downside Capture**: Performance asymmetry
- **Sortino Ratio**: Risk-adjusted return (downside focus)
- **Calmar Ratio**: Return per unit of max drawdown

In [ ]:
# Prepare data for PortfolioAnalyzer (needs DataFrame with Date column)
def create_analyzer_df(
    results_unhedged, results_quarterly, results_conditional, results_ladder
):
    """Merge strategy results into single DataFrame for analyzer."""
    df = pd.DataFrame(
        {
            "Date": results_unhedged.index,
            "Market": results_unhedged["Value"].values,
            "Unhedged": results_unhedged["Value"].values,
            "Quarterly": results_quarterly["Value"].values,
            "Conditional": results_conditional["Value"].values,
            "VIX-Ladder": results_ladder["Value"].values,
        }
    )
    return df


# Create analyzer DataFrames for each crash
dc_metrics_df = create_analyzer_df(
    results_unhedged_dc, results_quarterly_dc, results_conditional_dc, results_ladder_dc
)
gfc_metrics_df = create_analyzer_df(
    results_unhedged_gfc,
    results_quarterly_gfc,
    results_conditional_gfc,
    results_ladder_gfc,
)
covid_metrics_df = create_analyzer_df(
    results_unhedged_covid,
    results_quarterly_covid,
    results_conditional_covid,
    results_ladder_covid,
)

# Create analyzers (benchmark_col not benchmark_column!)
dc_analyzer = PortfolioAnalyzer(dc_metrics_df, benchmark_col="Market")
gfc_analyzer = PortfolioAnalyzer(gfc_metrics_df, benchmark_col="Market")
covid_analyzer = PortfolioAnalyzer(covid_metrics_df, benchmark_col="Market")

print("✓ Created analyzers for all three crashes")

In [ ]:
# Display comprehensive metric summaries
print("=" * 80)
print("DOT-COM BUBBLE (2000-2002) - Performance Metrics")
print("=" * 80)
print(dc_analyzer.get_summary())
print("\n" + "=" * 80)
print("GLOBAL FINANCIAL CRISIS (2008-2009) - Performance Metrics")
print("=" * 80)
print(gfc_analyzer.get_summary())
print("\n" + "=" * 80)
print("COVID-19 CRASH (2020) - Performance Metrics")
print("=" * 80)
print(covid_analyzer.get_summary())

In [ ]:
# Visualize Capture Ratios across crashes
fig, axes = plt.subplots(1, 3, figsize=(18, 5))
crashes = [("Dot-Com", dc_analyzer), ("GFC", gfc_analyzer), ("COVID", covid_analyzer)]
strategies = ["Unhedged", "Quarterly", "Conditional", "VIX-Ladder"]

for ax, (crash_name, analyzer) in zip(axes, crashes):
    upside_ratios = []
    downside_ratios = []

    for strategy in strategies:
        upside, downside = analyzer.calculate_capture_ratios(strategy)
        upside_ratios.append(upside * 100)  # Convert to percentage
        downside_ratios.append(downside * 100)

    x = np.arange(len(strategies))
    width = 0.35

    bars1 = ax.bar(
        x - width / 2,
        upside_ratios,
        width,
        label="Upside Capture",
        alpha=0.8,
        color="green",
    )
    bars2 = ax.bar(
        x + width / 2,
        downside_ratios,
        width,
        label="Downside Capture",
        alpha=0.8,
        color="red",
    )

    ax.set_xlabel("Strategy")
    ax.set_ylabel("Capture Ratio (%)")
    ax.set_title(f"{crash_name} Crash\nCapture Ratios")
    ax.set_xticks(x)
    ax.set_xticklabels(strategies, rotation=45, ha="right")
    ax.axhline(y=100, color="gray", linestyle="--", linewidth=0.8, alpha=0.5)
    ax.legend()
    ax.grid(axis="y", alpha=0.3)

    # Add value labels on bars
    for bars in [bars1, bars2]:
        for bar in bars:
            height = bar.get_height()
            ax.text(
                bar.get_x() + bar.get_width() / 2.0,
                height,
                f"{height:.0f}%",
                ha="center",
                va="bottom",
                fontsize=8,
            )

plt.tight_layout()
plt.show()

print("\n📊 Interpretation:")
print("• Lower downside capture = better protection during market crashes")
print("• VIX-Ladder should show significantly lower downside capture than unhedged")

In [ ]:
# Risk-Adjusted Returns: Sortino vs Beta scatter plots
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for ax, (crash_name, analyzer) in zip(axes, crashes):
    betas = []
    sortinos = []

    for strategy in strategies:
        beta = analyzer.calculate_beta(strategy)
        sortino = analyzer.calculate_sortino(strategy)
        betas.append(beta)
        sortinos.append(sortino)

    colors = ["red", "orange", "blue", "green"]
    for i, (strategy, beta, sortino) in enumerate(zip(strategies, betas, sortinos)):
        ax.scatter(beta, sortino, s=200, alpha=0.7, color=colors[i], label=strategy)
        ax.annotate(
            strategy,
            (beta, sortino),
            xytext=(5, 5),
            textcoords="offset points",
            fontsize=9,
        )

    ax.set_xlabel("Beta (Market Sensitivity)")
    ax.set_ylabel("Sortino Ratio")
    ax.set_title(f"{crash_name} Crash\nRisk-Return Profile")
    ax.axhline(y=0, color="gray", linestyle="--", linewidth=0.8, alpha=0.5)
    ax.axvline(x=1.0, color="gray", linestyle="--", linewidth=0.8, alpha=0.5)
    ax.grid(True, alpha=0.3)
    ax.legend(loc="best")

plt.tight_layout()
plt.show()

print("\n📊 Interpretation:")
print("• Higher Sortino = better risk-adjusted returns (downside focus)")
print("• Lower Beta = less market sensitivity during crashes")
print("• Best performers: high Sortino + low Beta (upper-left quadrant)")

In [ ]:
# Calmar Ratio Comparison (Return per unit of max drawdown)
fig, ax = plt.subplots(figsize=(12, 6))

crash_names = ["Dot-Com", "GFC", "COVID"]
x = np.arange(len(strategies))
width = 0.25

for i, (crash_name, analyzer) in enumerate(crashes):
    calmars = [analyzer.calculate_calmar(strategy) for strategy in strategies]
    offset = (i - 1) * width
    bars = ax.bar(x + offset, calmars, width, label=crash_name, alpha=0.8)

    # Add value labels
    for bar in bars:
        height = bar.get_height()
        ax.text(
            bar.get_x() + bar.get_width() / 2.0,
            height,
            f"{height:.2f}",
            ha="center",
            va="bottom",
            fontsize=8,
        )

ax.set_xlabel("Strategy")
ax.set_ylabel("Calmar Ratio")
ax.set_title(
    "Calmar Ratio Comparison Across Market Crashes\n(Higher = Better Return per Unit of Max Drawdown)"
)
ax.set_xticks(x)
ax.set_xticklabels(strategies)
ax.axhline(y=0, color="gray", linestyle="--", linewidth=0.8)
ax.legend()
ax.grid(axis="y", alpha=0.3)

plt.tight_layout()
plt.show()

print("\n📊 Key Insights:")
print("• Calmar Ratio = CAGR / |Max Drawdown|")
print("• Higher values = better return relative to worst drawdown")
print("• Negative values = negative returns during crash period")

## 🎯 Conclusions

The quantitative metrics confirm what the visual analysis showed:

**VIX-Ladder LP Performance:**
- **Beta**: Lower market sensitivity during crashes (protection when it matters)
- **Downside Capture**: Significantly reduced downside participation vs unhedged
- **Sortino Ratio**: Superior risk-adjusted returns (focuses on downside risk)
- **Calmar Ratio**: Better return per unit of maximum drawdown

The adaptive budget allocation (VIX-based + beta-adjusted) combined with the ladder structure (50% to catastrophic strikes) provides robust crash protection across different market environments.

## 📝 Summary: VIX-Ladder LP Strategy

### 🎯 Project Focus
This project implements **VIX-Ladder LP** as our core hedging approach, comparing it against baseline strategies:
- **Quarterly**: Fixed-schedule protective puts (industry standard)
- **Conditional**: Trigger-based hedging (activates on drawdowns/volatility spikes)
- **VIX-Ladder LP**: Our optimization-based approach with VIX-responsive budgeting and strike diversification

---

### ✅ **VIX-Ladder LP Strengths**

1. **VIX-Responsive Budgeting**
   - Budget scales automatically with market volatility
   - Formula: Budget = V₀ × 1% × (VIX/20) × max(1.0, β)
   - VIX=15: ~75% of base budget | VIX=30: ~150% of base budget
   - Higher beta portfolios get proportionally more protection

2. **Strike Ladder Diversification**
   - Protection distributed across multiple OTM levels:
     - Shallow (5-15% OTM): 5% of budget
     - Medium (15-25% OTM): 15% of budget  
     - Deep (25-40% OTM): 30% of budget
     - Catastrophic (40%+ OTM): 50% of budget
   - Reduces concentration risk vs single-strike hedging
   - Graduated payoffs during crashes

3. **LP Optimization**
   - Mathematically finds cost-optimal option mix
   - Subject to ladder budget allocation constraints
   - Minimizes total premium cost while ensuring diversification

---

### ⚠️ **Limitations**

1. **Not Crash-Proof**
   - Budget constraints limit maximum protection
   - Prolonged crashes (Dot-Com) still challenging
   - Reduces volatility ≠ eliminates crash risk

2. **Model Assumptions**
   - Relies on VIX as volatility proxy
   - Assumes options available at estimated prices
   - Budget formula (1% × VIX/20 × beta) is heuristic, not optimized

3. **Return Drag**
   - Premium costs compound over time
   - Tradeoff: stability vs absolute returns

---

### 📊 **Performance Characteristics**

**Best Use Cases:**
- **Fast crashes** (COVID): VIX-driven protection adapts quickly
- **High volatility periods**: Ladder ensures diversified coverage
- **Institutional portfolios**: Volatility reduction matters more than max returns

**When Quarterly/Conditional May Be Better:**
- **Low volatility bull markets**: Ladder may overpay for protection
- **Very long time horizons**: Simpler strategies have lower overhead
- **Constrained budgets**: Fixed quarterly costs easier to plan

---

### 🚀 **Key Innovation**

The VIX-Ladder LP combines two simple ideas:
1. **VIX-responsive budget**: Always buys protection (even when Gap G = 0), scaled to market conditions
2. **Ladder constraints**: Forces LP to diversify across strike levels instead of concentrating on "optimal" strikes

This creates a robust hedging strategy that:
- Adapts to changing volatility regimes
- Provides graduated protection across crash scenarios
- Remains cost-efficient through LP optimization

---

### 🎓 **For Presentation**

**Honest Framing:**
> "VIX-Ladder LP is an effective **volatility management** tool that outperforms baseline strategies in adaptive protection and cost efficiency. It reduces portfolio swings and provides diversified downside coverage, but is **not a crash-proof solution** — particularly for prolonged downturns."

**Key Contributions:**
- ✅ Mathematically rigorous LP framework with VIX-responsive budgeting
- ✅ Strike ladder diversification for graduated protection
- ✅ Empirical validation across 3 major crashes (Dot-Com, GFC, COVID)
- ✅ Honest assessment of strengths and limitations

**Acknowledged Limitations:**
- Budget constraints limit maximum protection
- Better for volatility management than extreme drawdowns
- Model relies on VIX accuracy and option pricing assumptions


In [ ]:
# Hyperparameter sweep: budget, strike, expiry
import itertools

# Choose one representative crash (COVID) to keep runtime reasonable
market = Market(ticker="^GSPC", start="2019-06-01", end="2021-06-30", fetch_vix=True)
initial = initial_value

budgets = [0.01, 0.015, 0.02]  # 1% to 2%
strike_ratios = [0.95, 0.92, 0.90]
expiry_days_list = [30, 60, 90]

rows = []
for budget, strike_ratio, expiry_days in itertools.product(
    budgets, strike_ratios, expiry_days_list
):
    # Build ladder allocations (keep same relative mix)
    default_ladder = [
        (0.05, 0.15, 0.05),
        (0.15, 0.25, 0.15),
        (0.25, 0.40, 0.30),
        (0.40, 1.00, 0.50),
    ]

    params = {
        "hedge_interval": 7,
        "expiry_days": expiry_days,
        "alpha": 0.05,
        "ladder_budget_allocations": default_ladder,
        "strike_density": 0.05,
        "transaction_cost_rate": 0.05,
        # hyperparameters to sweep
        "strike_ratio": strike_ratio,
        "base_budget_ratio": budget,
    }

    # Fresh portfolio per run
    portfolio = Portfolio(initial_value=initial, beta=1.0)
    results = run_simulation(market, portfolio, vix_ladder_strategy, params)

    # Prepare analyzer input (use results value as both portfolio and market proxy here)
    df = pd.DataFrame(
        {
            "Date": results.index,
            "Market": results["Value"].values,
            "VIX-Ladder": results["Value"].values,
        }
    )
    analyzer = PortfolioAnalyzer(df, benchmark_col="Market")

    summary = analyzer.get_summary()
    rows.append(
        {
            "Budget %": budget * 100,
            "Strike Ratio": strike_ratio,
            "Expiry Days": expiry_days,
            "Total Return (%)": float(summary.loc["VIX-Ladder", "Total Return (%)"]),
            "Max Drawdown (%)": float(summary.loc["VIX-Ladder", "Max Drawdown (%)"]),
            "Daily Vol (%)": float(summary.loc["VIX-Ladder", "Daily Vol (%)"]),
            "Sortino": float(analyzer.calculate_sortino("VIX-Ladder")),
            "Calmar": float(analyzer.calculate_calmar("VIX-Ladder")),
        }
    )

sweep_df = pd.DataFrame(rows)
sweep_df_sorted = sweep_df.sort_values(by=["Calmar", "Sortino"], ascending=False)
print("\nHyperparameter sweep (COVID period) — top configurations by Calmar/Sortino:\n")
print(sweep_df_sorted.head(10).to_string(index=False))